In [1]:
import numpy as np
import pandas as pd
from math import ceil
import pickle
with open('datapath.txt') as f:
    datapath=f.readlines()[0].rstrip()

In [3]:
%ls $datapath

cliente_tabla.csv.zip   test.csv.zip        train.csv*
producto_tabla.csv.zip  town_state.csv.zip  train.csv.zip


In [2]:
train=pd.read_csv(datapath+'train.csv',nrows=10,parse_dates=False)

In [3]:
dtypes=train.dtypes.to_dict()
dtypes['Semana']=np.uint8
dtypes

{'Agencia_ID': dtype('int64'),
 'Canal_ID': dtype('int64'),
 'Cliente_ID': dtype('int64'),
 'Demanda_uni_equil': dtype('int64'),
 'Dev_proxima': dtype('float64'),
 'Dev_uni_proxima': dtype('int64'),
 'Producto_ID': dtype('int64'),
 'Ruta_SAK': dtype('int64'),
 'Semana': numpy.uint8,
 'Venta_hoy': dtype('float64'),
 'Venta_uni_hoy': dtype('int64')}

In [4]:
yname='Demanda_uni_equil'

In [5]:
categoricals='Agencia_ID Canal_ID Ruta_SAK Cliente_ID Producto_ID'.split()

In [5]:
def addcounts(cnt,cnk,stcol):
    newcnt=[df_counts(cnk,stcol[i]) for i in xrange(len(stcol))]
    return [cnt[i].add(newcnt[i],axis='index',fill_value=0.0) for i in xrange(len(stcol))]

In [6]:
def df_counts(df,column):
    return df.groupby(column).size()

In [8]:
statcolumns='Agencia_ID Canal_ID Ruta_SAK Producto_ID Semana'.split()

In [9]:
numcolumns='Venta_uni_hoy Venta_hoy Dev_uni_proxima Dev_proxima Demanda_uni_equil'.split()

In [7]:
def add_stats(one,two):
    df=pd.DataFrame(columns=one.columns)
    df.loc['count']=one.loc['count']+two.loc['count']
    df.loc['mean']=(one.loc['count']*one.loc['mean'] + two.loc['count']*two.loc['mean'])/df.loc['count']
    df.loc['max']=np.maximum(one.loc['max'],two.loc['max'])
    df.loc['min']=np.minimum(one.loc['min'],two.loc['min'])
    return df
    

In [11]:
chunksz=100000
trainit=pd.read_csv(datapath+'train.csv',chunksize=chunksz,iterator=True,dtype=dtypes,parse_dates=False)

In [12]:
stats='count mean max min'.split()

In [13]:
#%%time
ichunk=0
for chunk in trainit:
    if ichunk==0:
        fullcount=[df_counts(chunk,col) for col in statcolumns]
        floatstats=chunk[numcolumns].describe().loc[stats]
        #sample=chunk.sample(frac=samplerate)
    else:
        fullcount=addcounts(fullcount,chunk,statcolumns)
        floatstats=add_stats(floatstats,chunk[numcolumns].describe().loc[stats])
    if ichunk % 100 == 0:
        print(ichunk)
        print map(lambda x: x.shape[0], fullcount)
    ichunk+=1

0
[3, 4, 151, 287, 1]
100
[499, 9, 2744, 1546, 1]
200
[552, 9, 3013, 1625, 2]
300
[552, 9, 3249, 1663, 3]
400
[552, 9, 3413, 1698, 4]
500
[552, 9, 3487, 1730, 5]
600
[552, 9, 3554, 1753, 6]
700
[552, 9, 3599, 1782, 7]


In [17]:
with open('catcount.pkl','w') as f:
    pickle.dump((floatstats,fullcount), f)

In [5]:
%ls

catcount.pkl          Cliente_ID_stats.py  Feature Stats Generation.ipynb
Cliente_ID_Stats.pkl  datapath.txt


I don't seem to have enough memory for `Cliente_ID`, so I had to run it from a raw python script, which I called `Cliente_ID_stats.py`. The results are saved in `Cliente_ID_Stats.pkl`.